## This tutorial presents how to simulate the full network + body movement
## Simulating body movement from network activity is managed by separate module called dynworm.body_sim
## The module takes neuronal voltage activities of full network from dynworm.network_sim, convert them into muscle activities, and muscle activities to movements of worm using the Viscoelastic Rod Model

## Step 1: Import necessary packages and modules

In [ ]:
%matplotlib inline

# external packages 

import os
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.animation as animation
import scipy.io as sio

# move directory to parent directory of main

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

# internal modules 

import dynworm as dw

## Step 2: Initialize neural parameters and connectivity
### Here we use the default neural parameters and connectomes

In [ ]:
dw.network_sim.initialize_params_neural()
dw.network_sim.initialize_connectivity()

## Step 3: Initialize body parameters
### Body parameters must be initialized prior to simulating the body.
### Once initialized, the parameters live in body_sim.params_obj_body, which is a global variable for dynworm.body_sim module  
### Similar to network physiological and connectivity matrices in dynworm.network_sim, body paramters can be modified by providing your own dictionary of body_parameters 
### In this tutorial, we use the default body parameters

In [ ]:
dw.body_sim.initialize_params_body()

## Step 4: Initialize input_vec and ablation vector
### We simulate the network with constant input stimuli into PLML and PLMR

In [ ]:
input_vec = np.zeros(dw.network_sim.params_obj_neural['N'])
ablation_mask = np.ones(dw.network_sim.params_obj_neural['N'], dtype = 'bool')

In [ ]:
# neurons_idx module has dictionary 'neurons_list' where you can easily reference the information about neurons of interest

neurons_list = dw.neurons_idx.neurons_list
neurons_list[276]

In [ ]:
# In this example, we are injecting 3.5nA constant current into PLMR (276th index) and PLML (278th index)

input_vec[276] = 0.35
input_vec[278] = 0.35

## Step 5: Run the simulation
### run_network_constinput in ce_network module simulates the network when given with input_mask vector
### t_start: start time of the simulation
### t_final: end time of the simulation
### t_delta: time resolution of simulation. This is not to be confused with integration step, which is determined internally
### input_vec: the input_vec vector with configured constant inputs
### ablation_mask: the ablation vector which define any neurons to be ablated

In [ ]:
result_dict_network = dw.network_sim.run_network_constinput(t_duration=10, 
                                                            input_vec = input_vec, ablation_mask=ablation_mask)

## Step 6: Simulate the body  
### Simplify provide result_dict from network_sim.run_network... functions to  dynworm.body_sim.solve_bodymodel function. The function also let you to choose the initial body orientation of the worm:
### xinit: Initial x_coordinate of the worm's head. Set to 0 by default.
### yinit: Initial y_coordinate of the worm's head. Set to 0 by default.
### orientation_angle: positive orientation angle --> counterclockwise rotation, negative orientation angle --> clockwise rotation. Set to 0 by default.

In [ ]:
import time

start = time.time()

result_dict_body = dw.body_sim.solve_bodymodel_ivp(result_dict_network = result_dict_network) 

end = time.time()

print(end - start)

### The function outputs result_dict_body, a dictionary object containing 
### "t": time vector for simulation
### "x": x coordinates for 93 body segments of shape (timepoints x body segments). The segments are ordered from anterior to posterior direction 
### "y": y coordinates for 93 body segments of shape (timepoints x body segments).
### "phi": dynamics of angles of 24 rod segments relative to horizontal plane

In [ ]:
result_dict_body['x'].shape

In [ ]:
x = result_dict_body['x']
y = result_dict_body['y']

In [ ]:
x_head = x[:, 0]
y_head = y[:, 0]

with sns.axes_style("white"):

    fig = plt.figure(figsize=(23,23))

    plt.imshow(swarm_image_bargmann, zorder=0, extent=[-200, 200, -200, 200])

    plt.plot(x_head, y_head, linewidth = 5, color = 'black')
    plt.xlim(-200, 200)
    plt.ylim(-200, 200)
    plt.show()

## Step 7: Produce body movement animation
### dynworm.body_sim module comes with an animation rederer "produce_animation" which produces a movie of simulated body
### Note that you need to install 'ffmpeg' package to compile the video. You can simply use "conda install" command in Anaconda prompt to include in Anaconda environment. 

### The function has 13 inputs
### x: x coordinates array with shape (timesteps, 93)
### y: y coordinates array with shape (timesteps, 93)
### filename: filename to be saved
### xmin: leftmost x coordinate for the video
### xmax: rightmost x coordinate for the video
### ymin: downmost y coordinate for the video
### ymax: upmost y coordinate for the video
### figsize_x: width of video
### figsize_y: height of video 
### color: color of the worm (default set to black)
### facecolor: background color (default set to white)
### axis: switch for drawing x-y axis (default set to on)
### timer: switch for timer that shows simulation time (default set to off)

In [ ]:
# For producing body animation, ensure that filename is in string format

dw.body_sim.produce_animation(x=x, y=y, filename='plm_3.5nA_grid_off', 
                              xmin=-120, xmax=80, ymin=-100, ymax=100, 
                              figsize_x=25, figsize_y=25, axis = 'on')